In [1]:
import numpy as np
import pandas as pd

# Assignment 1: Groupby

Can you return a table containing the top 10 stores by total transactions in the data?

Make sure they’re sorted from highest to lowest.

Thanks!

In [4]:
# Read in transactions data -- parse dates specified here for help with later problem

transactions = pd.read_csv("../../retail/transactions.csv", parse_dates=["date"])

transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [20]:
transactions.groupby("store_nbr")["transactions"].sum().sort_values(ascending=False).head(10)

store_nbr
44    7273093
47    6535810
45    6201115
46    5990113
3     5366350
48    5107785
8     4637971
49    4574103
50    4384444
11    3972488
Name: transactions, dtype: int64

# Assignment 2: Groupby Multiple Columns

Can you get me the total transactions by store and month? 

Sort the table from first month to last, then by highest transactions to lowest within each month. 


In [22]:
# helper code to extract month date part from date column

transactions["month"] = transactions["date"].dt.month

transactions.head()

,date,store_nbr,transactions,month
0,2013-01-01,25,770,1
1,2013-01-02,1,2111,1
2,2013-01-02,2,2358,1
3,2013-01-02,3,3487,1
4,2013-01-02,4,1922,1


In [26]:
(transactions
 .groupby(["store_nbr", "month"], as_index=False)["transactions"]
 .sum()
 .sort_values(["month", "transactions"], ascending=[True, False]))

,store_nbr,month,transactions
516,44,1,628438
552,47,1,568824
528,45,1,538370
540,46,1,522763
24,3,1,463260
...,...,...,...
383,32,12,86167
251,21,12,84128
503,42,12,76741
347,29,12,76627


# Assignment 3: Multi-Index DataFrames


Can you help me access rows and columns with multiple indices? I’ve been struggling with multi-index DataFrames.

Access:
* Grab Store 3, Month 1
* Then, select the column storing the mean of transactions

Fix:
* Drop the outer layer of the column Index
* Reset the row index so it is the default integer index

In [28]:
# Ross' grouped DataFrame code, run this first

grouped = (
    transactions.groupby(["store_nbr", "month"])
    .agg({"transactions": ["sum", "mean"]})
    .sort_values(by=["month", ("transactions", "sum")], ascending=[True, False])
)

In [30]:
# df grouped by store and month with two aggregation columns

grouped.head()

transactions             
                         sum         mean
store_nbr month                          
44        1           628438  4246.202703
47        1           568824  3843.405405
45        1           538370  3637.635135
46        1           522763  3532.182432
3         1           463260  3151.428571

In [32]:
grouped.loc[(3, 1)]

transactions  sum     463260.000000
              mean      3151.428571
Name: (3, 1), dtype: float64

In [42]:
grouped.loc[:, ("transactions", "mean")]

store_nbr  month
44         1        4246.202703
47         1        3843.405405
45         1        3637.635135
46         1        3532.182432
3          1        3151.428571
                       ...     
32         12        718.058333
21         12       1402.133333
42         12       1279.016667
29         12       1277.116667
22         12        844.166667
Name: (transactions, mean), Length: 641, dtype: float64

In [56]:
grouped.droplevel(0, axis=1).reset_index()

,store_nbr,month,sum,mean
0,44,1,628438,4246.202703
1,47,1,568824,3843.405405
2,45,1,538370,3637.635135
3,46,1,522763,3532.182432
4,3,1,463260,3151.428571
...,...,...,...,...
636,32,12,86167,718.058333
637,21,12,84128,1402.133333
638,42,12,76741,1279.016667
639,29,12,76627,1277.116667


In [60]:
grouped.droplevel(0, axis=1).reset_index()

,store_nbr,month,sum,mean
0,44,1,628438,4246.202703
1,47,1,568824,3843.405405
2,45,1,538370,3637.635135
3,46,1,522763,3532.182432
4,3,1,463260,3151.428571
...,...,...,...,...
636,32,12,86167,718.058333
637,21,12,84128,1402.133333
638,42,12,76741,1279.016667
639,29,12,76627,1277.116667


# Assignment 4: The Agg Method

Calculate the mean of target met by store, and the sum of bonuses to be paid to each store.

Sort them by highest to lowest bonus payout.

Then, do the same for day of week and month.

In [62]:
# Recreate table from section 3

transactions = transactions.assign(
    target_pct=transactions["transactions"] / 2500,
    met_target=(transactions["transactions"] / 2500) >= 1,
    bonus_payable=((transactions["transactions"] / 2500) >= 1) * 100,
    month=transactions.date.dt.month,
    day_of_week=transactions.date.dt.dayofweek,
)

transactions.head()

,date,store_nbr,transactions,month,target_pct,met_target,bonus_payable,day_of_week
0,2013-01-01,25,770,1,0.3080,False,0,1
1,2013-01-02,1,2111,1,0.8444,False,0,2
2,2013-01-02,2,2358,1,0.9432,False,0,2
3,2013-01-02,3,3487,1,1.3948,True,100,2
4,2013-01-02,4,1922,1,0.7688,False,0,2


In [68]:
(transactions
 .groupby("store_nbr")
 .agg(
     total_bonuses = ("bonus_payable", "sum"), 
     avg_target_met = ("met_target", "mean"))
 .sort_values("total_bonuses", ascending=False))

,total_bonuses,avg_target_met
store_nbr,,
47,167600,0.999404
44,167500,0.998807
45,167300,0.997615
3,167300,0.998210
46,165900,0.989267
8,148900,0.888425
48,115800,0.690519
49,106900,0.637448
50,76000,0.453190


In [70]:
(transactions
 .groupby("day_of_week")
 .agg(
     total_bonuses = ("bonus_payable", "sum"), 
     avg_target_met = ("met_target", "mean"))
 .sort_values("total_bonuses", ascending=False))

,total_bonuses,avg_target_met
day_of_week,,
5,266400,0.222204
6,241700,0.204001
4,213000,0.179007
0,191600,0.160214
2,191000,0.160572
1,175500,0.146299
3,169100,0.142077


In [72]:
(transactions
 .groupby("month")
 .agg(
     total_bonuses = ("bonus_payable", "sum"), 
     avg_target_met = ("met_target", "mean"))
 .sort_values("total_bonuses", ascending=False))

,total_bonuses,avg_target_met
month,,
12,154100,0.255640
5,131800,0.170792
3,130400,0.169461
4,129700,0.174469
7,126300,0.162486
2,121700,0.174230
6,121700,0.161706
8,120800,0.174189
1,119600,0.163723


# Assignment 4: Transform

Calculate the mean of transactions by store number and day of week while keeping row numbers. 

Then compare the performance of each row to its day of week average. (difference between transactions and daily avg)

In [104]:
transactions = (transactions
                .assign(
                    avg_transactions_day_of_week = (transactions
                                                    .groupby(["store_nbr", "day_of_week"])["transactions"]
                                                    .transform("mean")), 
                    diff_from_day_avg = transactions["transactions"] - transactions["avg_transactions_day_of_week"]
                )
)

In [102]:
transactions

,date,store_nbr,transactions,month,target_pct,met_target,bonus_payable,day_of_week,avg_transactions_day_of_week,diff_from_day_avg
0,2013-01-01,25,770,1,0.3080,False,0,1,740.245690,29.754310
1,2013-01-02,1,2111,1,0.8444,False,0,2,1870.782427,240.217573
2,2013-01-02,2,2358,1,0.9432,False,0,2,1952.652720,405.347280
3,2013-01-02,3,3487,1,1.3948,True,100,2,3142.682008,344.317992
4,2013-01-02,4,1922,1,0.7688,False,0,2,1499.569038,422.430962
...,...,...,...,...,...,...,...,...,...,...
83483,2017-08-15,50,2804,8,1.1216,True,100,1,2342.410788,461.589212
83484,2017-08-15,51,1573,8,0.6292,False,0,1,1548.448133,24.551867
83485,2017-08-15,52,2255,8,0.9020,False,0,1,1892.588235,362.411765
83486,2017-08-15,53,932,8,0.3728,False,0,1,877.214286,54.785714


# Assignment 5: Pivot

Pivot transactions with store number as index, columns day of week, with the sum of bonus payable as cells.

Filter to stores that had a non-zero bonus payable and create a heatmap.

Then unpivot (melt) the table so we have one row for each store and day of the week with the corresponding total owed. 


In [ ]:
# Use transactions table (ok if includes columns from assignment 4 or not)

transactions.head()